## Preprocessing
The preprocessing pipelines are defined in this notebook. Throughout the project, two pipelines are mainly used, the pipeline that cleans the data, so that it can be used to extract other features, and the pipeline that extracts the `signal` and `label` for a neural network.

In [ ]:
from sia import Preprocessing
from sia.io import Metadata, read_edf, read_csv, write_csv
from sia.preprocessors import neurokit

### Cleaning Pipeline
The pipeline defined beneath uses the default preprocessor of Neurokit to clean the data. Furthermore, the data with a low quality is also discarded.

In [ ]:
Preprocessing() \
    .data(
        read_edf(
            './data/raw/*.edf', 
            Metadata('./data/raw/TimeStamps_Merged.txt').on_regex(r'[0-9]{5}')
        )
    ) \
    .process(neurokit()) \
    .filter(lambda ECG_Quality: [quality > .25 for quality in ECG_Quality]) \
    .to(write_csv('./data/cleaned/[0-9]{5}.csv'))

### Neural Network 
For the neural network, only the signal and the label is extracted and saved in another directory.

In [ ]:
Preprocessing() \
    .data(read_csv('./data/cleaned/*.csv', columns=['ECG_Clean', 'category'])) \
    .rename({'ECG_Clean': 'signal', 'category': 'label'}) \
    .to(write_csv('./data/signal/[0-9]{5}.csv'))